# Certified Finetuning of a Classifier on the OCT-MNIST Dataset

In [1]:
%load_ext autoreload
%autoreload 2
import os
import torch
import tqdm
import abstract_gradient_training as agt
from abstract_gradient_training import AGTConfig
from abstract_gradient_training import certified_training_utils as ct_utils
from models.deepmind import DeepMindSmall 
from datasets import oct_mnist
from models.robust_regularizer import gradient_interval_regularizer


## Test the robustness of a non-robustly pre-trained classifier

In [2]:
device = torch.device("cuda:0")
_, dl_test = oct_mnist.get_dataloaders(1000, exclude_classes=[2], balanced=True)
standard_model = DeepMindSmall(1, 1).to(device)
standard_model.load_state_dict(torch.load(".models/medmnist.ckpt"))
params_l, params_n, params_u = ct_utils.get_parameters(standard_model)
epsilon = 0.0005
test_batch, test_labels = next(iter(dl_test))
accs = agt.test_metrics.test_accuracy(
    params_l, params_n, params_u, test_batch, test_labels, standard_model, ct_utils.propagate_conv_layers, epsilon
)
accs = ", ".join([f"{a:.2f}" for a in accs])

print(f"Accuracy of non-robustly trained classifier on test set with epsilon={epsilon}: [{accs}]")

Accuracy of non-robustly trained classifier on test set with epsilon=0.0005: [0.04, 0.95, 1.00]


## Pre-train the model

Exclude class 2 (Drusen) from the pretraining.

In [7]:
# set up pre-training
torch.manual_seed(1)
pretrain_batchsize = 100
pretrain_n_epochs = 20
pretrain_learning_rate = 0.001
pretrain_epsilon = 0.001
pretrain_model_epsilon = 0.0
pretrain_reg_strength = 1e-3
model_path = f".models/medmnist_robust_eps{pretrain_epsilon}_alpha{pretrain_reg_strength}_meps{pretrain_model_epsilon}.ckpt"

In [8]:
# define model, dataset and optimizer
model = DeepMindSmall(1, 1)
dl_pretrain, _ = oct_mnist.get_dataloaders(pretrain_batchsize, exclude_classes=[2], balanced=True)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=pretrain_learning_rate)
model = model.to(device)

In [9]:
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))
else:  # pre-train the model
    progress_bar = tqdm.trange(pretrain_n_epochs, desc="Epoch")
    for epoch in progress_bar:
        for i, (x, u) in enumerate(dl_pretrain):
            # Forward pass
            u, x = u.to(device), x.to(device)
            output = model(x)
            bce_loss = criterion(output.squeeze().float(), u.squeeze().float())
            regularization = gradient_interval_regularizer(
                model, x, u, "binary_cross_entropy", pretrain_epsilon, pretrain_model_epsilon
            )
            loss = bce_loss + pretrain_reg_strength * regularization
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if i % 100 == 0:
                progress_bar.set_postfix(loss=loss.item(), bce_loss=bce_loss.item())
    # save the model
    with open(model_path, "wb") as file:
        torch.save(model.state_dict(), file)

Epoch: 100%|██████████| 20/20 [03:51<00:00, 11.56s/it, bce_loss=0.227, loss=0.265]


### Test the robustness of the model pre-trained with the gradient interval regularization term

In [10]:
_, dl_test = oct_mnist.get_dataloaders(1000, exclude_classes=[2], balanced=True)
params_l, params_n, params_u = ct_utils.get_parameters(model)
epsilon = 0.0005
test_batch, test_labels = next(iter(dl_test))
accs = agt.test_metrics.test_accuracy(
    params_l, params_n, params_u, test_batch, test_labels, model, ct_utils.propagate_conv_layers, epsilon
)
accs = ", ".join([f"{a:.2f}" for a in accs])

print(f"Accuracy of robustly trained classifier on test set with epsilon={epsilon}: [{accs}]")

Accuracy of robustly trained classifier on test set with epsilon=0.0005: [0.93, 0.94, 0.95]


## Fine-tune the model

Include all classes, only allowing class 2 (Drusen) to be potentially poisoned.

In [12]:
# set up fine-tuning parameters
clean_batchsize = 7000
drusen_batchsize = 3000
test_batchsize = 1000

config = AGTConfig(
    fragsize = 2000,
    learning_rate = 0.6,
    n_epochs = 2,
    k_poison = 50,
    epsilon = 0.01,
    forward_bound = "interval",
    device = "cuda:0",
    backward_bound = "interval",
    loss = "binary_cross_entropy",
    lr_decay=2.0,
    lr_min=0.001,
)

In [13]:
# get dataloaders
dl_clean, dl_test_clean = oct_mnist.get_dataloaders(clean_batchsize, test_batchsize, exclude_classes=[2])
dl_drusen, dl_test_drusen = oct_mnist.get_dataloaders(drusen_batchsize, test_batchsize, exclude_classes=[0, 1, 3])
_, dl_test_all = oct_mnist.get_dataloaders(clean_batchsize, test_batchsize)

In [14]:
# evaluate the pre-trained model
param_n, param_l, param_u = ct_utils.get_parameters(model)
drusen_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_drusen)), model, ct_utils.propagate_conv_layers)
clean_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_clean)), model, ct_utils.propagate_conv_layers)
all_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_all)), model, ct_utils.propagate_conv_layers)

print("=========== Pre-trained model accuracy ===========")
print(f"Class 2 (Drusen) : nominal = {drusen_acc[1]:.2g}")
print(f"Classes 0, 1, 3  : nominal = {clean_acc[1]:.2g}")
print(f"All Classes      : nominal = {all_acc[1]:.2g}")

=========== Pre-trained model accuracy ===========
Class 2 (Drusen) : nominal = 0.43
Classes 0, 1, 3  : nominal = 0.94
All Classes      : nominal = 0.81


In [15]:
# fine-tune the model using abstract gradient training (keeping the convolutional layers fixed)
param_l, param_n, param_u = agt.poison_certified_training(
    model, config, dl_drusen, dl_test_drusen, dl_clean=dl_clean, transform=ct_utils.propagate_conv_layers
)

[AGT] [INFO    ] [17:14:37] =================== Starting Poison Certified Training ===================
[AGT] [INFO    ] [17:14:37] Starting epoch 1
[AGT] [INFO    ] [17:14:37] Training batch 1: Network eval bounds=(0.43, 0.43, 0.43), W0 Bound=0.0 
[AGT] [INFO    ] [17:14:38] Training batch 2: Network eval bounds=(1   , 1   , 1   ), W0 Bound=0.07 
[AGT] [INFO    ] [17:14:39] Starting epoch 2
[AGT] [INFO    ] [17:14:39] Training batch 3: Network eval bounds=(1   , 1   , 1   ), W0 Bound=0.16 
[AGT] [INFO    ] [17:14:40] Training batch 4: Network eval bounds=(0.71, 1   , 1   ), W0 Bound=0.271 
[AGT] [INFO    ] [17:14:41] Final network eval: Network eval bounds=(0.33, 0.84, 1   ), W0 Bound=0.429 
[AGT] [INFO    ] [17:14:41] =================== Finished Poison Certified Training ===================


In [16]:
# evaluate the fine-tuned model
drusen_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_drusen)), model, ct_utils.propagate_conv_layers)
clean_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_clean)), model, ct_utils.propagate_conv_layers)
all_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, *next(iter(dl_test_all)), model, ct_utils.propagate_conv_layers)

print("=========== Fine-tuned model accuracy + bounds ===========")
print(f"Class 2 (Drusen) : nominal = {drusen_acc[1]:.2g}, certified bound = {drusen_acc[0]:.2g}")
print(f"Classes 0, 1, 3  : nominal = {clean_acc[1]:.2g}, certified bound = {clean_acc[0]:.2g}")
print(f"All Classes      : nominal = {all_acc[1]:.2g}, certified bound = {all_acc[0]:.2g}")

=========== Fine-tuned model accuracy + bounds ===========
Class 2 (Drusen) : nominal = 0.84, certified bound = 0.33
Classes 0, 1, 3  : nominal = 0.87, certified bound = 0.6
All Classes      : nominal = 0.86, certified bound = 0.54
